In [1]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
filename = '../../datasets/flickr8k/Flickr8k_text/Flickr_8k.trainImages.txt'

In [3]:
def save_model(model, filename):
    model_json = model.to_json()
    with open(filename + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(filename + ".h5")
    print("Saved model to disk")

In [4]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [5]:
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    for line in doc.split('\n'):
        if len(line) < 1:
            continue
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

In [6]:
def load_clean_descriptions(filename, dataset):
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]
        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = list()
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            descriptions[image_id].append(desc)
    return descriptions

In [7]:
def load_photo_features(filename, dataset):
    all_features = load(open(filename, 'rb'))
    features = {k: all_features[k] for k in dataset}
    return features

In [8]:
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

In [9]:
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [10]:
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

In [11]:
def create_sequences(tokenizer, max_length, desc_list, photo):
    X1, X2, y = list(), list(), list()
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(photo)
            X2.append(in_seq)
            y.append(out_seq)
    return array(X1), array(X2), array(y)

In [12]:
def data_generator(descriptions, photos, tokenizer, max_length):
    while 1:
        for key, desc_list in descriptions.items():
            photo = photos[key][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo)
            yield [[in_img, in_seq], out_word]

In [13]:
def define_model(vocab_size, max_length):
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    decoder3 = Dropout(0.5)(decoder2)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.summary()
    return model

In [14]:
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 6000


In [15]:
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=6000


In [16]:
train_features = load_photo_features('features.pkl', train)
print('Photos: train=%d' % len(train_features))

Photos: train=6000


In [17]:
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 7579


In [18]:
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 34


In [19]:
model = define_model(vocab_size, max_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 256)      1940224     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
__________________________________________________________________________________________________
dropout_2 

## Train model

In [20]:
epochs = 10
steps = len(train_descriptions)
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    save_model(model, './model_weights/model_' + str(i))

Epoch 1/1
6000/6000 [==============================] - 948s 158ms/step - loss: 4.5719
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 928s 155ms/step - loss: 3.7786
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 1076s 179ms/step - loss: 3.5129
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 900s 150ms/step - loss: 3.3538
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 2610s 435ms/step - loss: 3.2507
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 1239s 206ms/step - loss: 3.1762
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 813s 136ms/step - loss: 3.1214
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 829s 138ms/step - loss: 3.0792
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 853s 142ms/step - loss: 3.0428
Saved model to disk
Epoch 1/1
6000/6000 [======================

In [21]:
epochs = 2
steps = len(train_descriptions)
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    save_model(model, './model_weights/model_' + str(i+10))

Epoch 1/1
6000/6000 [==============================] - 981s 164ms/step - loss: 2.9946
Saved model to disk
Epoch 1/1
6000/6000 [==============================] - 1299s 216ms/step - loss: 2.9769
Saved model to disk


## Train model with validation set

In [ ]:
validation_filename = '../../datasets/flickr8k/Flickr8k_text/Flickr_8k.devImages.txt'

In [ ]:
validation = load_set(validation_filename)
print('Dataset: %d' % len(validation))

In [ ]:
validation_descriptions = load_clean_descriptions('descriptions.txt', validation)
print('Descriptions: validation=%d' % len(validation_descriptions))

In [ ]:
validation_features = load_photo_features('features.pkl', validation)
print('Photos: validation=%d' % len(validation_features))

In [ ]:
epochs = 10
steps = len(train_descriptions)
validation_steps=len(validation_descriptions)
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    validation_generator = data_generator(validation_descriptions, validation_features, tokenizer, max_length)#check tokenizer
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1,validation_data=validation_generator,validation_steps=validation_steps)
    save_model(model, './model_weights/model_' + str(i))